In [79]:
import pandas as pd
import numpy as np

In [80]:
from itertools import chain
from collections import Counter
from functools import partial

In [81]:
train = pd.read_csv('dt_train1.txt', sep='\t')
test = pd.read_csv('dt_test1.txt', sep='\t')

In [82]:
y_label, *_ = list(set(train.columns) - set(test.columns))

In [83]:
df, labels = pd.concat([train, test]), dict()
for column in df:
    df[column], labels[column] = pd.factorize(df[column])
    # labels[column] = labels[column].get_values()

In [84]:
x_train = df.iloc[:len(train)].drop(y_label, axis=1)
y_train = df.iloc[:len(train)][y_label]
x_test = df.iloc[len(train):].drop(y_label, axis=1)

In [85]:
class DecisionTree():
    def __init__(self, metric):
        self.tree = None
        self._metric = metric
    
    def _terminal(self, train):
        return Counter(train[:, -1]).most_common()[0][0]
    
    def _build(self, train, depth=1):
        # gini index
        _metric = lambda groups: sum([(1.-sum([(v/len(g))**2 for k, v in Counter(g[:, -1]).items()])) * (len(g)/len(list(chain(*groups)))) for g in filter(np.any, groups)])
        # split data where value lower than v and else
        _split = lambda i, v: (train[np.where(train[:, i] < v)], train[np.where(train[:, i] >= v)])
        # _split and calc value using _metric
        _apply = np.vectorize(lambda v, i: _metric(_split(i, v)))
        # get index of _applyed minimum
        _mini = lambda uni, idx, i: idx[_apply(uni, i).argmin()]
        
        m = np.apply_along_axis(lambda i: _mini(*np.unique(train[:, i], return_index=True), i), 1, np.array([np.arange(self._c-1)]).T)
        i, j = min(enumerate(m), key=lambda t: _metric(_split(t[0], train[t[1]][t[0]])))
        l, r = _split(i, train[j][i])
        
        node = {
            'index': i,
            'value': train[j][i],
            'left': l,
            'right': r
        }
        
        if not len(l) or not len(r):
            node['left'] = node['right'] = self._terminal(np.concatenate([l, r]))
        elif depth >= self.max_depth:
            node['left'], node['right'] = self._terminal(l), self._terminal(r)
        else:
            node['left'] = self._terminal(l) if len(l) <= self.min_size else self._build(l, depth+1)
            node['right'] = self._terminal(r) if len(r) <= self.min_size else self._build(r, depth+1)
        return node
    
    def fit(self, X, y, max_depth=32, min_size=.0):
        self.max_depth = max_depth
        self.min_size = min_size
        
        train = np.concatenate([X, np.array([y]).T], axis=1)
        self._r, self._c = train.shape
        self.tree = self._build(train)

    def _predict(self, node, x):
        tar = node['left'] if x[node['index']] < node['value'] else node['right']
        return self._predict(tar, x) if isinstance(tar, dict) else tar
    
    def predict(self, X):
        return np.apply_along_axis(partial(self._predict, self.tree), 1, X)

In [86]:
classifier = DecisionTree(0)

In [87]:
classifier.fit(x_train, y_train, 16, 0)

In [88]:
test[y_label] = pd.Series(map(lambda y: labels[y_label][y], classifier.predict(x_test)))

In [89]:
test.to_csv('dt_result1.txt', sep='\t', index=None)

In [90]:
answer = pd.read_csv('dt_answer1.txt', sep='\t')

In [91]:
sum(test[y_label] == answer[y_label]), len(test[y_label])

(326, 346)